In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [2]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df['v2']=df['v2'].str.lower()

In [4]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"go until jurong point, crazy.. available only ...",NaN,NaN,NaN
1,ham,ok lar... joking wif u oni...,NaN,NaN,NaN
2,spam,free entry in 2 a wkly comp to win fa cup fina...,NaN,NaN,NaN
3,ham,u dun say so early hor... u c already then say...,NaN,NaN,NaN
4,ham,"nah i don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
df=df[['v1','v2']]

In [6]:
df.head()

,v1,v2
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [7]:
import string 
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
# text cleaning
for i in string.punctuation:
    df['v2']=df['v2'].str.replace(i," ")
df.head()
df['v2']=df['v2'].str.replace("   ","")
df['v2']=df['v2'].str.replace("  ","")
df.head()

,v1,v2
0,ham,go until jurong pointcrazyavailable only in bu...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don t think he goes to usfhe lives aroun...


In [9]:
print(df['v2'][0])

go until jurong pointcrazyavailable only in bugis n great world la e buffet cine there got amore wat


In [10]:
# converting text to numeric data 
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
# Tokenization
df['v2'].str.split()

0       [go, until, jurong, pointcrazyavailable, only,...
1                          [ok, lar, joking, wif, u, oni]
2       [free, entry, in, 2, a, wkly, comp, to, win, f...
3       [u, dun, say, so, early, hor, u, c, already, t...
4       [nah, i, don, t, think, he, goes, to, usfhe, l...
                              ...                        
5567    [this, is, the, 2nd, time, we, have, tried, 2,...
5568           [will, ìb, going, to, esplanade, fr, home]
5569    [pity, was, in, mood, for, thatsoany, other, s...
5570    [the, guy, did, some, bitching, but, i, acted,...
5571                       [roflits, true, to, its, name]
Name: v2, Length: 5572, dtype: object

In [12]:
# Remove stopwords-> remove useless words like is,the,and
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
df['v2']=df['v2'].apply(
    lambda x: " ".join(word for word in x.split() if word not in ENGLISH_STOP_WORDS)
)

In [13]:
df['v2']

0       jurong pointcrazyavailable bugis n great world...
1                                 ok lar joking wif u oni
2       free entry 2 wkly comp win fa cup final tkts 2...
3                             u dun say early hor u c say
4                        nah don t think goes usfhe lives
                              ...                        
5567    2nd time tried 2 contact uu won å£750 pound pr...
5568                           ìb going esplanade fr home
5569                      pity mood thatsoany suggestions
5570    guy did bitching acted like d interested buyin...
5571                                         roflits true
Name: v2, Length: 5572, dtype: object

In [14]:
# Lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['v2']=df['v2'].apply(
    lambda x: " ".join(lemmatizer.lemmatize(word) for word in x.split())
)

In [15]:
df['v2'][0]

'jurong pointcrazyavailable bugis n great world la e buffet cine got amore wat'

In [16]:
# convert Text -> Number (BoW)
cv = CountVectorizer(max_features = 3000)
x = cv.fit_transform(df['v2'])

In [17]:
y = df['v1'].map({'ham':0, 'spam':1})

In [18]:
df['v1'][0]

'ham'

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state = 42)

In [20]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x_train , y_train)

MultinomialNB()

In [21]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.97847533632287
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.93      0.91      0.92       150

    accuracy                           0.98      1115
   macro avg       0.96      0.95      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [22]:
model.score(x_test,y_test)

0.97847533632287